In [1]:
from pathlib import Path

import os
from pathlib import Path
import argparse

import torch
import tqdm
import yaml
from attrdict import AttrDict

from modules.comp.comp_d_net_pl import *
from modules.mono.depth_net_pl import *
from modules.mv.mv_depth_net_pl import *
from utils.data_utils import *
from utils.localization_utils import *


from torch.utils.data import DataLoader
import lightning as Lightning


/cluster/home/wueestm/anaconda3/envs/f3loc/lib/python3.8/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
net_type = "d"#"comp"#

# dataset = "gibson_f"
# #dataset_path = "/cluster/scratch/wueestm/gibson/Gibson_Floorplan_Localization_Dataset"
# dataset_path= "/cluster/project/cvg/data/gibson/Gibson_Floorplan_Localization_Dataset"

dataset = "hge_customized_cropped"
dataset_path = "/cluster/project/cvg/data/lamar/HGE_customized_cropped"

In [3]:
# get device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("======= USING DEVICE : ", device, " =======")

======= USING DEVICE :  cpu  =======


In [4]:
# parameters
L = 0  # number of the source frames
D = 128  # number of depth planes
d_min = 0.1  # minimum depth
d_max = 15.0  # maximum depth
d_hyp = -0.2  # depth transform (uniform sampling in d**d_hyp)
#F_W = 3 / 8  # camera intrinsic, focal length / image width
#trans_thresh = 0.005  # translation threshold (variance) if using comp_s

In [5]:
add_rp = (
    True  # whether use roll and pitch angle augmentation, only used in training
)
roll = 0.00001  # maximum roll augmentation in radian
pitch = 0.00001  # maximum pitch augmentation in radian

In [6]:
# paths
dataset_dir = os.path.join(dataset_path, dataset)
depth_dir = dataset_dir
#log_dir = ckpt_path
desdf_path = os.path.join(dataset_path, "desdf")

if (net_type == "d") & (dataset == "hge_customized_cropped"):
    depth_suffix = "depth90"
elif net_type == "d":
    depth_suffix = "depth40"
else:
    depth_suffix = "depth160"

In [7]:
# instantiate dataset
split_file = os.path.join(dataset_dir, "split.yaml")
with open(split_file, "r") as f:
    split = AttrDict(yaml.safe_load(f))

In [8]:
# Define data
train_dataset = GridSeqDataset_hge_customized_cropped_gravity_align(
    dataset_dir,
    split.train,
    L=L,
    depth_dir=depth_dir,
    depth_suffix=depth_suffix,
    add_rp=add_rp,
    roll=roll,
    pitch=pitch,
)


val_dataset = GridSeqDataset_hge_customized_cropped_gravity_align(
    dataset_dir,
    split.val,
    L=L,
    depth_dir=depth_dir,
    depth_suffix=depth_suffix,
    add_rp=add_rp,
    roll=roll,
    pitch=pitch,
)

self.scene_names:  ('ios_2022-01-12_15.15.53_000', 'ios_2022-01-18_17.05.03_000', 'ios_2022-01-18_17.10.39_000', 'ios_2022-01-20_16.52.33_001', 'ios_2022-01-25_14.34.24_002', 'ios_2022-01-25_14.57.49_000', 'ios_2022-01-25_15.13.54_000', 'ios_2022-06-13_10.45.07_000', 'ios_2022-06-13_15.59.36_000', 'ios_2022-06-14_17.12.28_000', 'ios_2022-06-30_15.55.53_000', 'ios_2022-07-01_15.18.09_000')
scene:  ios_2022-01-12_15.15.53_000


scene:  ios_2022-01-18_17.05.03_000
scene:  ios_2022-01-18_17.10.39_000
scene:  ios_2022-01-20_16.52.33_001
scene:  ios_2022-01-25_14.34.24_002
scene:  ios_2022-01-25_14.57.49_000
scene:  ios_2022-01-25_15.13.54_000
scene:  ios_2022-06-13_10.45.07_000
scene:  ios_2022-06-13_15.59.36_000
scene:  ios_2022-06-14_17.12.28_000
scene:  ios_2022-06-30_15.55.53_000
scene:  ios_2022-07-01_15.18.09_000
self.scene_names:  ('ios_2022-07-01_15.45.08_000', 'ios_2022-07-01_15.58.10_000', 'ios_2022-01-18_17.10.39_000')
scene:  ios_2022-07-01_15.45.08_000
scene:  ios_2022-07-01_15.58.10_000
scene:  ios_2022-01-18_17.10.39_000


In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)


In [10]:
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [11]:
# Define model 
model = depth_net_pl(d_min=d_min, d_max=d_max, d_hyp=d_hyp, D=D)


/cluster/home/wueestm/anaconda3/envs/f3loc/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/cluster/home/wueestm/anaconda3/envs/f3loc/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
from lightning.pytorch.loggers import TensorBoardLogger  # Import TensorBoardLogger

# Setup TensorBoard logger
logger = TensorBoardLogger("tb_logs", name="my_model")

In [13]:
# Train the model
#trainer = Lightning.Trainer(max_steps=10, max_epochs=1, enable_checkpointing=True)
trainer = Lightning.Trainer(max_epochs=1, enable_checkpointing=True, logger=logger)
#trainer = Lightning.Trainer(max_epochs=100, enable_checkpointing=True, logger=logger)
#trainer = Lightning.Trainer(max_steps=8, max_epochs=1, enable_checkpointing=True)
#trainer = Lightning.Trainer(max_epochs=8, enable_checkpointing=True)
#trainer = Lightning.Trainer(fast_dev_run=10, enable_checkpointing=True)
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)


/cluster/home/wueestm/anaconda3/envs/f3loc/lib/python3.8/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cluster/home/wueestm/anaconda3/envs/f3loc/lib/pytho ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type      | Params
--------------------------------------
0 | encoder | depth_net | 25.9 M
--------------------------------------
25.9 M    Trainable params
0         Non-trainable params
25.9 M    Total params
103.726   Total estimated model params size (MB)


Sanity Checking: |                                                                                            …

/cluster/home/wueestm/anaconda3/envs/f3loc/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/cluster/home/wueestm/anaconda3/envs/f3loc/lib/python3.8/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
